# Exercise 4b: Riemann Solvers for Burgers' Equation

## Learning Objectives

By completing this exercise, you will:
- Understand the conservative form of hyperbolic PDEs
- Implement a Riemann solver (Rusanov/Local Lax-Friedrichs)
- Compute interface fluxes on a staggered grid
- Compare diffusivity of Riemann solvers vs. the Lax method

---

## Introduction

We continue with Burgers' equation from [Exercise 3](https://github.com/AST-Course/AST5110/blob/main/notebooks/ex_3.ipynb):

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = 0   \tag{1}$$

**Setup:**
- Domain: $x \in [-1.4, 2.0]$
- Initial condition:
$$u(x,t=0) = A\left[\tanh\left(\frac{x+x_c}{W}\right)-\tanh\left(\frac{x-x_c}{W}\right)\right]   \tag{2}$$
- Parameters: $A = 0.02$, $x_c = 0.70$, $W = 0.1$
- Final time: $t_f = 100$

In this exercise, we implement a **Riemann solver** approach. See the [wiki: Riemann Solvers](https://github.com/AST-Course/AST5110/wiki/Riemann-solvers) for theoretical background.

**Implementation:** Fill in the function `evolv_Rie_uadv_burgers` in [`nm_lib/nm_ex/nm_lib_ex_4.py`](https://github.com/AST-Course/nm_lib/tree/master/nm_lib/nm_ex/nm_lib_ex_4.py).

---

## The Riemann Solver Algorithm

### Step 1: Compute Left and Right States

Compute the left and right states at each interface:
- $u_L = u_i$ (left of interface $i+1/2$)
- $u_R = u_{i+1}$ (right of interface $i+1/2$)

### Step 2: Compute the Flux

Rewrite eq. (1) in **conservative form** to identify the flux $F(u)$.

From the left and right states, compute:
- $F_L = F(u_L)$
- $F_R = F(u_R)$

**Implementation:** Use `Rie_flux` in `nm_lib/nm_ex/nm_lib_ex_4.py`.

### Step 3: Compute the Wave Speed

The propagation speed is the derivative of the flux with respect to $u$:
$$v_a = \max\left( \left|\frac{\partial F}{\partial u}\right|_L, \left|\frac{\partial F}{\partial u}\right|_R \right)$$

This will be the derivatie of the flux with respect to variable $u$ in absolute values and select the largest between the left and right shells

**Implementation:** Use `Rie_va` in `nm_lib/nm_ex/nm_lib_ex_4.py`.

> See the [wiki: Riemann Solvers](https://github.com/AST-Course/AST5110/wiki/Riemann-solvers) for details on the propagation speed derivation.

### Step 4: Compute Interface Fluxes (Rusanov)

The **Rusanov flux** (Local Lax-Friedrichs) at interface $i+1/2$:

$$F^*_{i+1/2} = \frac{F_R + F_L}{2} - \frac{v_a}{2}(u_R - u_L)$$

> **Note:** $F^*_{i+1/2}$ is defined at **half-grid points** (staggered).

**Implementation:** Use `Rie_interface_flux` in `nm_lib/nm_ex/nm_lib_ex_4.py`.

### Step 5: Time Advance

Update the solution using flux differences:

$$u^{n+1}_i = u^n_i - \frac{\Delta t}{\Delta x}\left(F^*_{i+1/2} - F^*_{i-1/2}\right)$$

**Implementation:** Use `evolv_Rie_uadv_burgers` in `nm_lib/nm_ex/nm_lib_ex_4.py`.

**Question:** What is $\Delta t$ given the phase speed $v_a$? (CFL condition)

---

## Part 2: Comparison with Lax Method

### Task 2.1: Compare Diffusivity

**Questions:**
1. Is the Riemann solver more or less diffusive than the Lax method?
2. Find the analytical solution and compare with both numerical methods
3. Where and when does each method perform better or worse?

---

## Background: Riemann Solver Variants

> **Note:** Different Riemann solvers use different approximations for $F^*$ and the wave speed $v_a$, but they all follow the same algorithmic steps:
> - **Rusanov (LLF):** Uses maximum wave speed (most diffusive)
> - **HLL:** Uses two wave speeds (less diffusive)
> - **HLLC:** Includes contact discontinuity (even less diffusive)
> - **Roe:** Uses linearized Jacobian (least diffusive, but can fail)

---

## Hints

<details>
<summary><b>Hint 1: Conservative form of Burgers' equation</b></summary>

The conservative form is:
$$\frac{\partial u}{\partial t} + \frac{\partial F}{\partial x} = 0$$

where the flux is $F(u) = \frac{u^2}{2}$.
</details>

<details>
<summary><b>Hint 2: Wave speed for Burgers' equation</b></summary>

Since $F(u) = u^2/2$, we have:
$$\frac{\partial F}{\partial u} = u$$

So $v_a = \max(|u_L|, |u_R|)$.
</details>

<details>
<summary><b>Hint 3: CFL condition with Riemann solver</b></summary>

The time step is limited by the maximum wave speed:
$$\Delta t = \text{CFL} \cdot \frac{\Delta x}{\max(v_a)}$$

with CFL $\leq 1$ for stability.
</details>